In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import polars as plot
import glob
from importlib import reload


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
import seasonality.seasonalityfunctions as sf
import seasonality.onset_demise as od

In [6]:
reload(od)
reload(sf)

<module 'seasonality.seasonalityfunctions' from '/Users/gbromley/code/PrecipitationSeasonality/seasonality/seasonalityfunctions.py'>

In [ ]:
test_bombardi = xr.open_dataset('/Users/gbromley/data/b17_cpc_onset-demise/onset.wet.season.CPC_UNI.1999.nc', decode_cf = False)

In [ ]:
### processing CPC data 


cpc_files = glob.glob('/Users/gbromley/data/cpc_uni_global/precip.1999.nc')
cpc_raw = xr.open_dataset(cpc_files[0])

### Setting coordinate data for next step 
lat = np.arange(-90, 90, 0.5)
lon = np.arange(0, 360, 0.5)
time = cpc_raw['time'].data

### transposing dimensions so the world is right side up 

flipped_cpc = np.flip(cpc_raw['precip'].values, axis=1)
### cpc latitudes are reversed so that they are decreasing instead of increasing. The following code reverses it. 
cpc_pr = xr.DataArray(flipped_cpc, coords=[('time', time), ('lat', lat), ('lon', lon)], name='precipitation')

### Selecting out CONUS from the global data.
conus = {'lat': slice(25,50), 'lon': slice(360-125, 360-66)}
cpc_conus = cpc_pr.sel(lat = conus['lat'], lon = conus['lon'])

In [ ]:
annual_cycle = sf.calc_annual_cycle(cpc_conus)

In [ ]:
start_wet = od.B17_analysis_start(annual_cycle)

In [ ]:
start_wet.plot()

In [ ]:
onset = od.onset_B17(cpc_conus, start_wet)

In [7]:
sf.generate_p_da(5, 2015)

In [4]:
test

In [ ]:
B17_onset = test_bombardi['DOY'].sel(conus).squeeze()
B17_onset = xr.where(B17_onset == -999., np.nan, B17_onset)

In [ ]:
B17_onset.plot()

In [ ]:
onset.plot()

In [ ]:
onset.values

In [ ]:
(B17_onset - onset.squeeze()).plot()

In [ ]:
def find_sequence(arr, sequence):
    if len(sequence) == 0:
        return -1

    # Masks for each element in the sequence
    masks = [arr == val for val in sequence]

    # Find indices where the first element of the sequence occurs
    first_elem_indices = np.where(masks[0])[0]

    for idx in first_elem_indices:
        if all((idx + i < len(arr) and masks[i][idx + i]) for i in range(len(sequence))):
            return idx

    return -1

In [ ]:
mask = (seq[i-2] < 0,)

In [ ]:
arr = np.array([1, -3, 2, -7, 5, 6, 7, 8, 9, -4, -3, -2])
sequence = [5, 3, 2]
index = find_sequence(arr, sequence)
index

In [ ]:
array = np.array([1, 2, 3])
variable = xr.Variable("x", array)
data_array = xr.DataArray(variable, [("x", -array)])
dataset = xr.Dataset({"y": variable}, {"x": -array})

In [ ]:
dataset

In [ ]:
x = np.linspace(-10, 10, 400)
arr = x**3

# Calculate the first derivative
dy = 3 * x**2

In [ ]:
plt.plot(dy)

In [ ]:
f = np.array([1, 2, 4, 7, 11, 16], dtype=float)

np.gradient(f)


np.gradient(f, 2)


In [ ]:
sf.find_ddt_onset(np.sign(np.diff(np.gradient(arr))))

In [ ]:
x = np.linspace(-10, 10, 400)
arr = x**3
sf.find_ddt_onset(arr)


In [ ]:
array = np.array([1,2,3])
isinstance(array, np.ndarray)

In [ ]:
time_array = pd.date_range('2000-01-01', freq='D', periods = 5)

In [ ]:
time_array

In [ ]:
isinstance(time_array, pd.DatetimeIndex)

In [ ]:
isinstance(time_array[0], np.datetime64[ns])

In [25]:
def generate(years = 5, start = 2015):
    DAYS_IN_YEAR = 366
    
    """
    Summary:
        Generates synthetic precipitation data for the testing of functions, and learning.

    Raises:
        TypeError: Years needs to be an int.
        TypeError: start needs to be an int.

    Returns:
        DataArray: Xarray data array is returned.
    """

    if not isinstance(years, int):
        raise TypeError('Year argument needs to be an integer!')

    if not isinstance(start, int):
        raise TypeError('Start argument needs to be an integer')

# Create a date range for the specified number of years
    dates = pd.date_range(start=f'{start}-01-01', periods=years*DAYS_IN_YEAR)

    # Create a baseline seasonal cycle of precipitation (sinusoidal)
    # This simulates higher precipitation in certain months
    seasonal_cycle = np.sin(2 * np.pi * dates.dayofyear / DAYS_IN_YEAR)

    # Add a stochastic component using Gaussian noise
    stochastic_component = np.random.normal(0, 1, size=len(dates))

    # Combine the seasonal and stochastic components to get the final precipitation data
    # Adjust the amplitude and mean to realistic values (e.g., mean=3, amplitude=2)
    mean_precipitation = 3
    amplitude = 2
    precipitation_data = mean_precipitation + amplitude * seasonal_cycle + stochastic_component

    # Ensure all precipitation values are non-negative
    precipitation_data = np.maximum(precipitation_data, 0)

    
    # Create a DataFrame
    da = xr.DataArray(
        data=precipitation_data,
        dims=["time"],
        coords=dict(time=dates),
        attrs=dict(
            description="Synthetically generated precipitation data (fake).",
            units="mm",
        ),
    )
    
    

    return da


In [35]:
len(generate())

1830

In [45]:
reload(sf)
da = sf.generate_p_da()

In [46]:
da

<xarray.DataArray (time: 1825)>
array([3.506876, 3.566184, 2.740081, ..., 3.96836 , 1.601097, 2.444137])
Coordinates:
  * time     (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2019-12-31
Attributes:
    description:  Synthetically generated precipitation data (fake).
    units:        mm

In [38]:
da = da.sel(time=~((da.time.dt.month == 2) & (da.time.dt.day == 29)))

In [49]:
test = sf.generate_p_da(5, 2016)

In [84]:
sf.calc_annual_cycle(test)

<xarray.DataArray (dayofyear: 365)>
array([2.39964488, 2.94348435, 2.69078452, 3.6834908 , 2.79470701,
       3.01905117, 2.27796467, 3.09855866, 2.57007244, 2.98041297,
       2.90059406, 3.7417606 , 3.83196061, 3.92032251, 3.0960502 ,
       2.87471541, 4.66553211, 3.27129487, 4.59125021, 2.94871044,
       4.12553892, 3.79912702, 4.12851836, 2.94368923, 3.91171071,
       4.80514867, 3.91658693, 4.29570425, 3.71770139, 3.40684945,
       4.850189  , 3.40363577, 3.98969766, 4.81252087, 3.31814663,
       4.05089113, 4.39486112, 4.40011758, 4.80415231, 3.4832027 ,
       4.72333982, 4.13800713, 3.52048524, 3.75532351, 4.08120358,
       3.64813883, 3.82790809, 3.69754394, 4.67960721, 5.43215907,
       4.06703555, 4.95681364, 4.28129279, 5.1428977 , 5.22435462,
       4.75011777, 5.25855273, 4.57684347, 5.12559666, 4.53749528,
       5.20580983, 4.84978662, 5.76424102, 4.50482603, 4.93828671,
       4.82622662, 5.16940586, 4.73330328, 4.89882227, 5.24731754,
       4.77115395, 5.56402165, 5.00324877, 5.04545804, 4.87503615,
       5.37016823, 4.70725929, 5.70329677, 5.05514478, 5.23846501,
       4.89440076, 5.06777429, 5.26842132, 4.40957015, 4.80054643,
       5.11698366, 5.53297965, 6.40067748, 5.23939681, 4.73339525,
       4.73548286, 4.8470501 , 5.06685471, 5.80448727, 5.58128038,
       5.00628194, 5.28707224, 5.54279512, 4.91440157, 4.94863313,
...
       1.30643659, 1.10832061, 0.75932849, 1.70731986, 0.92742599,
       1.28068659, 1.23236994, 0.63514542, 0.45206892, 0.53718813,
       1.71810808, 1.81983781, 1.19871729, 1.72199317, 0.90792738,
       2.06785081, 1.66823627, 1.60167179, 0.58523058, 0.95325139,
       1.0950669 , 1.31597964, 1.13696842, 1.8757196 , 1.93033996,
       1.3399034 , 1.77736173, 1.53546068, 0.99467348, 2.51242182,
       0.87224285, 1.40819508, 0.38277174, 1.2670216 , 1.10088507,
       1.64868573, 1.7071206 , 1.66308961, 1.07745457, 0.96974434,
       1.01335535, 1.27604618, 1.7004927 , 0.96997588, 1.72540759,
       2.06495281, 1.34362038, 1.65561091, 1.79874131, 1.96262653,
       1.43337957, 1.40443418, 1.62311587, 0.66451853, 2.37732728,
       1.68377767, 1.27102371, 1.5454939 , 1.5575818 , 2.75493257,
       1.22611727, 1.60119597, 1.63274773, 2.35676792, 2.09800151,
       3.14934248, 2.53826913, 1.59278254, 1.77682158, 1.26530623,
       1.99341856, 2.52776391, 2.63517742, 2.17699289, 2.69446547,
       1.8518097 , 2.38104625, 2.08694652, 1.93726529, 1.20852583,
       2.34938768, 2.16894341, 2.06640659, 2.39621628, 2.75404804,
       2.1829252 , 2.01895903, 3.17615059, 2.58782194, 3.08443471,
       2.52684028, 2.84819992, 2.92621787, 2.50200256, 3.22461855,
       2.9945694 , 2.17532768, 2.38170124, 3.03447532, 1.71861136])
Coordinates:
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 360 361 362 363 364 365 366
Attributes:
    description:  Synthetically generated precipitation data (fake).
    units:        mm

In [71]:
s = pd.date_range(start='2016-01-01',periods = 5*365+2, freq='D')


In [81]:
s[(s.day == 29 & s.month==2)]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [82]:
start_year = 2016
end_year = 2020

s = pd.date_range(start=f'{start_year}-01-01', end=f'{end_year}-12-31', freq='D')

s = s[~((s.month == 2) & (s.day == 29))]

array([], dtype=float64)